In [20]:
import pandas as pd
uni = pd.read_csv("Uniclass2015.csv")

# Fill down the Group, Sub group, and Section columns to propagate the last valid observation forward
uni['Group'] = uni['Group'].fillna(method='ffill')
uni['Sub group'] = uni['Sub group'].fillna(method='ffill')
uni['Section'] = uni['Section'].fillna(method='ffill')

# Define a function to find the parent code
def find_parent(code):
    parts = code.split('_')
    if len(parts) > 1:
        return '_'.join(parts[:-1])
    return ''

# Create the Parent column
uni['Parent'] = uni['Code'].apply(find_parent)

# Create the final table
tableuni = uni[['Code', 'Parent', 'Title']]
tableuni.columns = ["Uni Code","Parent", "Uni Title"]
tableuni

,Uni Code,Parent,Uni Title
0,Ac,,Activities
1,Co,,Complexes
2,EF,,Elements/ functions
3,En,,Entities
4,FI,,Form of Information
...,...,...,...
15370,Ro_70_70_37,Ro_70_70,Highways operative
15371,Ro_70_70_68,Ro_70_70,Rail labourer
15372,Ro_70_70_70,Ro_70_70,Rail track layer
15373,Ro_70_70_72,Ro_70_70,Rail traction technician


In [2]:
from rank_bm25 import BM25Okapi
import pandas as pd
# Your titles from the first set
from nltk.stem import PorterStemmer

# Initialize the lemmatizer
stemmer = PorterStemmer()

# Example list of titles that you would replace with your actual data

# Lemmatize each title
uni = pd.read_csv("Uniclass2015.csv")
ebkp = pd.read_csv("eBKP_processed.csv")

# Tokenize both the titles and the IFC componentsstemmer.stem(word)
tokenized_titles = [[stemmer.stem(word) for word in title.lower().split()] for title in uni.Title]
tokenized_ebkp = [[stemmer.stem(word) for word in m.lower().split()] for m in ebkp.Translated_Text]

# Create a BM25 object with the IFC components
bm25 = BM25Okapi(tokenized_ebkp)
bests = []
for title in tokenized_titles:
    scores = bm25.get_scores(title)
    if sum(scores) > 0:
        best_match_index = scores.argmax()
        bests.append(ebkp.code[best_match_index])
    else:
        bests.append(pd.NA)  # Use "NA" if no relevant match is found

# Create DataFrame for eBKP and Uni mapping
ebkp_uni = pd.DataFrame({
    "eBKP": bests,
    "Uni Code": uni.Code.tolist(),
    "Uni Title": uni.Title.tolist(),
})


In [21]:
ebkp_uni

,eBKP,Uni Code,Uni Title
0,<NA>,Ac,Activities
1,<NA>,Co,Complexes
2,<NA>,EF,Elements/ functions
3,<NA>,En,Entities
4,C04.04,FI,Form of Information
...,...,...,...
15370,V03.03,Ro_70_70_37,Highways operative
15371,<NA>,Ro_70_70_68,Rail labourer
15372,I04.03,Ro_70_70_70,Rail track layer
15373,<NA>,Ro_70_70_72,Rail traction technician


In [3]:
from rank_bm25 import BM25Okapi
import pandas as pd
# Your titles from the first set
from nltk.stem import PorterStemmer

# Initialize the lemmatizer
stemmer = PorterStemmer()

# Example list of titles that you would replace with your actual data

# Lemmatize each title
uni = pd.read_csv("Uniclass2015.csv")
mf = pd.read_csv("MF_processed.csv")

# Tokenize both the titles and the IFC componentsstemmer.stem(word)
tokenized_titles = [[stemmer.stem(word) for word in title.lower().split()] for title in uni.Title]
tokenized_mf = [[stemmer.stem(word) for word in m.lower().split()] for m in mf.label]

# Create a BM25 object with the IFC components
bm25 = BM25Okapi(tokenized_mf)
bests = []
for title in tokenized_titles:
    scores = bm25.get_scores(title)
    if sum(scores) > 0:
        best_match_index = scores.argmax()
        bests.append(mf.code[best_match_index])
    else:
        bests.append(pd.NA)  # Use "NA" if no relevant match is found


mf_uni = pd.DataFrame({
    "MF": bests,
    "Uni Code": uni.Code.tolist(),
    "Uni Title": uni.Title.tolist(),
    })
mf_uni

,MF,Uni Code,Uni Title
0,01 90 00,Ac,Activities
1,<NA>,Co,Complexes
2,08 34 00,EF,Elements/ functions
3,00 52 96,En,Entities
4,28 05 45.15,FI,Form of Information
...,...,...,...
15370,01 78 23.13,Ro_70_70_37,Highways operative
15371,05 52 00,Ro_70_70_68,Rail labourer
15372,34 11 00,Ro_70_70_70,Rail track layer
15373,34 24 00,Ro_70_70_72,Rail traction technician


In [4]:
from rank_bm25 import BM25Okapi
import pandas as pd
# Your titles from the first set
from nltk.stem import PorterStemmer

# Initialize the lemmatizer
stemmer = PorterStemmer()

# Example list of titles that you would replace with your actual data

# Lemmatize each title
uni = pd.read_csv("Uniclass2015.csv")
ifc = pd.read_csv("IFC_processed.csv")

# Tokenize both the titles and the IFC componentsstemmer.stem(word)
tokenized_titles = [[stemmer.stem(word) for word in title.lower().split()] for title in uni.Title]
tokenized_ifc = [[stemmer.stem(word) for word in ifc.lower().split()] for ifc in ifc.IFC]

# Create a BM25 object with the IFC components
bm25 = BM25Okapi(tokenized_ifc)
bests = []
for title in tokenized_titles:
    scores = bm25.get_scores(title)
    if sum(scores) > 0:
        best_match_index = scores.argmax()
        bests.append(ifc.raw[best_match_index])
    else:
        bests.append(pd.NA)  # Use "NA" if no relevant match is found

ifc_uni = pd.DataFrame({
    "IFC": bests,
    "Uni Code": uni.Code.tolist(),
    "Uni Title": uni.Title.tolist(),
    })

In [5]:
table = pd.merge(tableuni, ifc_uni, how='left', on=["Uni Code", "Uni Title"])
table = pd.merge(table, mf_uni, how='left', on=["Uni Code", "Uni Title"])
table = pd.merge(table, ebkp_uni, how='left', on=["Uni Code", "Uni Title"])
table

,Uni Code,Parent,Uni Title,IFC,MF,eBKP
0,Ac,,Activities,IfcCooledBeam.ACTIVE,01 90 00,<NA>
1,Co,,Complexes,IfcBuildingElementProxy.COMPLEX,<NA>,<NA>
2,EF,,Elements/ functions,<NA>,08 34 00,<NA>
3,En,,Entities,<NA>,00 52 96,<NA>
4,FI,,Form of Information,IfcRoof.FREEFORM,28 05 45.15,C04.04
...,...,...,...,...,...,...
15370,Ro_70_70_37,Ro_70_70,Highways operative,<NA>,01 78 23.13,V03.03
15371,Ro_70_70_68,Ro_70_70,Rail labourer,IfcRail,05 52 00,<NA>
15372,Ro_70_70_70,Ro_70_70,Rail track layer,IfcRail,34 11 00,I04.03
15373,Ro_70_70_72,Ro_70_70,Rail traction technician,IfcRail,34 24 00,<NA>


In [16]:
import pandas as pd
import numpy as np
# Assuming df is your DataFrame with the following columns: 'Uni Code', 'Parent', 'Uni Title', 'IFC', 'MF', 'eBKP'
# Let's say df is already loaded with your data
df = table
# Function to determine the level of hierarchy based on 'Uni Code'
def get_hierarchy_level(uni_code):
    return uni_code.count('_')

# Apply the function to create a new 'Level' column
df['Level'] = df['Uni Code'].apply(get_hierarchy_level)

# Sort the DataFrame by 'Uni Code' to ensure parents are before children
df = df.sort_values(by='Uni Code')

# Find the maximum level for each 'Parent'
max_levels = df.groupby('Parent')['Level'].max()

# Merge the max_levels into the original DataFrame
df = df.merge(max_levels.rename('Max Level'), left_on='Parent', right_index=True)

# Now, fill the 'IFC', 'MF', and 'eBKP' columns only for the lowest level
# Create a mask for rows which are at the lowest level of their branch
max_level = max(df['Level'])

mask = df['Level'] == max_level

df = df[mask]
df.drop(columns=['Level', 'Max Level'], inplace=True)


table = df
table

,Uni Code,Parent,Uni Title,IFC,MF,eBKP
2840,Pr_15_31_04_02,Pr_15_31_04,Acid neutralization products,IfcProduct,02 52 19,H01.01
2841,Pr_15_31_04_06,Pr_15_31_04,Biocidal cleaning solutions,<NA>,01 74 13,G06.03
2842,Pr_15_31_04_10,Pr_15_31_04,Chemical absorbent products,IfcProduct,02 52 00,H01.01
2843,Pr_15_31_04_11,Pr_15_31_04,Chemical cleaning gels and liquids,IfcLiquidTerminal,40 24 16,G06.03
2844,Pr_15_31_04_12,Pr_15_31_04,Chemical poultices,<NA>,02 52 00,<NA>
...,...,...,...,...,...,...
15005,TE_70_80_25_32,TE_70_80_25,Fuse finder kits,<NA>,26 28 13,<NA>
15006,TE_70_80_25_42,TE_70_80_25,Illuminance meters,IfcFlowMeter,28 21 21,<NA>
15007,TE_70_80_25_44,TE_70_80_25,Impedance meters,IfcFlowMeter,11 12 23,<NA>
15008,TE_70_80_25_76,TE_70_80_25,Scopemeters,<NA>,<NA>,<NA>


In [19]:
from neo4j import GraphDatabase

# Function to create Cypher queries from a pandas DataFrame
def create_cypher_queries_from_dataframe(df):
    queries = []
    for index, row in df.iterrows():
        # Ensure valid Cypher labels for the component node (labels cannot start with a number or contain special characters)
        component_label = row['Uni Code'].replace(' ', '_').replace('.', '_')

        # Create the main component node
        query = f"MERGE (component:{component_label} {{name: '{row['Uni Title']}'}}) "

        # Add relationship to parent if it exists
        if pd.notna(row['Parent']):
            # Ensure valid Cypher labels for the parent node
            parent_label = 'P' + row['Parent'].replace(' ', '_').replace('.', '_')
            query += f"MERGE (parent:{parent_label} {{name: '{row['Parent']}'}}) WITH component MERGE (component)-[:IS_A]->(parent)  "

        # Add relationship to IFC component if it exists
        if pd.notna(row['IFC']):
            ifc_label = 'Ifc_' + row['IFC'].replace(' ', '_').replace('.', '_')
            query += f"WITH component MATCH (i:{ifc_label} {{name: '{row['IFC']}'}}) WITH component  MERGE (component)<-[:RELATED]->(i)  "

        # Add relationship to MF component if it exists
        if pd.notna(row['MF']):
            mf_label = 'MF_' + row['MF'].replace(' ', '_').replace('.', '_')
            query += f"WITH component MATCH (m:{mf_label} {{name: '{row['MF']}'}}) WITH component  MERGE (component)<-[:RELATED]->(m)  "

        # Add relationship to eBKP component if it exists
        if pd.notna(row['eBKP']):
            eBKP_label = 'eBKP_' + row['eBKP'].replace(' ', '_').replace('.', '_')
            query += f"MATCH (e:{eBKP_label} {{name: '{row['eBKP']}'}}) WITH component  MERGE (component)<-[:RELATED]->(e)  "
        
        queries.append(query)
    return queries

# Function to execute Cypher queries in a batch
def run_cypher_queries_in_batch(driver, queries):
    with driver.session() as session:
        for query in queries:
            session.run(query)


# Create queries from DataFrame
queries = create_cypher_queries_from_dataframe(table)

URI = "neo4j+ssc://d4e7c69a.databases.neo4j.io"
AUTH = ("neo4j", "8gGwVhSx2-ycIPiPGOWejHAhufieq2XOOrkOAizxa1E")
driver = GraphDatabase.driver(URI, auth=AUTH)

# Run the queries in a batch
run_cypher_queries_in_batch(driver, queries[0:10])

# Close the driver connection when done
driver.close()


CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: WITH is required between MERGE and MATCH (line 1, column 396 (offset: 395))
"MERGE (component:Pr_15_31_04_02 {name: 'Acid neutralization products'}) MERGE (parent:PPr_15_31_04 {name: 'Pr_15_31_04'}) WITH component MERGE (component)-[:IS_A]->(parent)  WITH component MATCH (i:Ifc_IfcProduct {name: 'IfcProduct'}) WITH component  MERGE (component)<-[:RELATED]->(i)  WITH component MATCH (m:MF_02_52_19 {name: '02 52 19'}) WITH component  MERGE (component)<-[:RELATED]->(m)  MATCH (e:eBKP_H01_01 {name: 'H01.01'}) WITH component  MERGE (component)<-[:RELATED]->(e)"
                                                                                                                                                                                                                                                                                                                                                                          